# TensorFlow2教程-自定义回调

自定义回调是一个很好用的工具，可以在训练，评估和推理期间自定义模型的行为，包括读取/更改keras模型等。

In [1]:
from __future__ import absolute_import, division,print_function, unicode_literals
import tensorflow as tf

/home/doit/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## 1 Keras回调简介
在Kreas中，Callback是一个python类，旨在被子类化以提供特定功能，并在训练的各阶段（包括每个batch/epoch的开始和结束），以及测试中调用一组方法。

我们可以通过回调列表，传递回调方法，在训练/评估/推断的不同阶段调用回调方法。

构建一个模型

In [2]:
def get_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(1, activation = 'linear', input_dim = 784))
    model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.1), loss='mean_squared_error', metrics=['mae'])
    return model

导入数据

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255

定义一个简单的自定义回调，以跟踪每批数据的开始和结束。

In [4]:
import datetime

class MyCustomCallback(tf.keras.callbacks.Callback):

    def on_train_batch_begin(self, batch, logs=None):
        print('Training: batch {} begins at {}'.format(batch, datetime.datetime.now().time()))

    def on_train_batch_end(self, batch, logs=None):
        print('Training: batch {} ends at {}'.format(batch, datetime.datetime.now().time()))

    def on_test_batch_begin(self, batch, logs=None):
        print('Evaluating: batch {} begins at {}'.format(batch, datetime.datetime.now().time()))

    def on_test_batch_end(self, batch, logs=None):
        print('Evaluating: batch {} ends at {}'.format(batch, datetime.datetime.now().time()))

在训练时传入回调函数

In [5]:
model = get_model()
_ = model.fit(x_train, y_train,
          batch_size=64,
          epochs=1,
          steps_per_epoch=5,
          verbose=0,
          callbacks=[MyCustomCallback()])

Training: batch 0 begins at 14:47:16.854006
Training: batch 0 ends at 14:47:19.215169
Training: batch 1 begins at 14:47:19.215812
Training: batch 1 ends at 14:47:19.219226
Training: batch 2 begins at 14:47:19.219737
Training: batch 2 ends at 14:47:19.222817
Training: batch 3 begins at 14:47:19.223307
Training: batch 3 ends at 14:47:19.226198
Training: batch 4 begins at 14:47:19.226991
Training: batch 4 ends at 14:47:19.229930


### 1.1 以下方法会调用回调函数
fit(), fit_generator()
训练或使用迭代数据进行训练。

evaluate(), evaluate_generator()
评估或使用迭代数据进行评估。

predict(), predict_generator()
预测或使用迭代数据进行预测。

In [6]:
_ = model.evaluate(x_test, y_test, batch_size=128, verbose=0, steps=5,
          callbacks=[MyCustomCallback()])

Evaluating: batch 0 begins at 14:52:23.452381
Evaluating: batch 0 ends at 14:52:23.492404
Evaluating: batch 1 begins at 14:52:23.492663
Evaluating: batch 1 ends at 14:52:23.493493
Evaluating: batch 2 begins at 14:52:23.493606
Evaluating: batch 2 ends at 14:52:23.494279
Evaluating: batch 3 begins at 14:52:23.494388
Evaluating: batch 3 ends at 14:52:23.495022
Evaluating: batch 4 begins at 14:52:23.495128
Evaluating: batch 4 ends at 14:52:23.495710


## 2 回调方法概述
### 2.1 训练/测试/预测的常用方法
为了进行训练，测试和预测，提供了以下方法来替代。

on_(train|test|predict)_begin(self, logs=None)
在fit/ evaluate/ predict开始时调用。

on_(train|test|predict)_end(self, logs=None)
在fit/ evaluate/ predict结束时调用。

on_(train|test|predict)_batch_begin(self, batch, logs=None)
在培训/测试/预测期间处理批次之前立即调用。在此方法中，logs是带有batch和size可用键的字典，代表当前批次号和批次大小。

on_(train|test|predict)_batch_end(self, batch, logs=None)
在培训/测试/预测批次结束时调用。在此方法中，logs是一个包含状态指标结果的字典。

### 2.2 训练时特定方法
另外，为了进行培训，提供以下内容。

on_epoch_begin（self，epoch，logs = None）
在训练期间的开始时调用。

on_epoch_end（self，epoch，logs = None）
在训练期间的末尾调用。

### 2.3 logsdict的用法
该logs字典包含损loss，已经每个batch和epoch的结束时的所有指标。示例包括loss和平均绝对误差。

In [7]:
class LossAndErrorPrintingCallback(tf.keras.callbacks.Callback):

    def on_train_batch_end(self, batch, logs=None):
        print('For batch {}, loss is {:7.2f}.'.format(batch, logs['loss']))

    def on_test_batch_end(self, batch, logs=None):
        print('For batch {}, loss is {:7.2f}.'.format(batch, logs['loss']))

    def on_epoch_end(self, epoch, logs=None):
        print('The average loss for epoch {} is {:7.2f} and mean absolute error is {:7.2f}.'.format(epoch, logs['loss'], logs['mae']))

model = get_model()
_ = model.fit(x_train, y_train,
          batch_size=64,
          steps_per_epoch=5,
          epochs=3,
          verbose=0,
          callbacks=[LossAndErrorPrintingCallback()])

For batch 0, loss is   31.99.
For batch 1, loss is  888.84.
For batch 2, loss is   20.06.
For batch 3, loss is    8.30.
For batch 4, loss is   10.34.
The average loss for epoch 0 is  191.91 and mean absolute error is    8.17.
For batch 0, loss is    6.29.
For batch 1, loss is    7.78.
For batch 2, loss is    5.60.
For batch 3, loss is    6.66.
For batch 4, loss is    5.95.
The average loss for epoch 1 is    6.45 and mean absolute error is    2.06.
For batch 0, loss is    6.19.
For batch 1, loss is    5.54.
For batch 2, loss is    7.04.
For batch 3, loss is   10.74.
For batch 4, loss is   15.87.
The average loss for epoch 2 is    9.07 and mean absolute error is    2.46.


同样，可以在evaluate时调用回调。

In [9]:
_ = model.evaluate(x_test, y_test, batch_size=128, verbose=0, steps=20,
          callbacks=[LossAndErrorPrintingCallback()])

For batch 0, loss is   23.48.
For batch 1, loss is   20.85.
For batch 2, loss is   22.11.
For batch 3, loss is   24.52.
For batch 4, loss is   26.07.
For batch 5, loss is   23.15.
For batch 6, loss is   23.08.
For batch 7, loss is   21.37.
For batch 8, loss is   22.78.
For batch 9, loss is   22.38.
For batch 10, loss is   24.63.
For batch 11, loss is   26.51.
For batch 12, loss is   23.29.
For batch 13, loss is   25.12.
For batch 14, loss is   25.20.
For batch 15, loss is   22.27.
For batch 16, loss is   26.29.
For batch 17, loss is   25.76.
For batch 18, loss is   24.94.
For batch 19, loss is   23.94.


## 3 keras回调示例
## 3.1 以最小的损失尽早停止
第一个示例展示了Callback通过达到最小损失时更改属性model.stop_training（布尔值），停止Keras训练。用户可以提供一个参数patience来指定训练最终停止之前应该等待多少个时期。

注：tf.keras.callbacks.EarlyStopping 提供了更完整，更通用的实现。

In [17]:
import numpy as np
class EarlyStoppingAtMinLoss(tf.keras.callbacks.Callback):
    def __init__(self, patience=0):
        super(EarlyStoppingAtMinLoss, self).__init__()
        self.patience = patience
        self.best_weights = None  # loss最低时的权重
    def on_train_begin(self, logs=None):
        # loss不再下降时等待的轮数
        self.wait = 0
        # 停止时的轮数
        self.stopped_epoch = 0
        # 开始时的最优loss
        self.best = np.Inf
    
    def on_epoch_end(self, epoch, logs=None):
        current = logs.get('loss')
        if np.less(current, self.best):
            self.best = current
            self.wait = 0
            # 最佳权重
            self.best_weights = self.model.get_weights()
        else:
            self.wait += 1
            if self.wait >= self.patience:
                self.stopped_epoch = epoch
                self.model.stop_training = True
                print('导入当前最佳模型')
                self.model.set_weights(self.best_weights)
    def on_train_end(self, logs=None):
        if self.stopped_epoch > 0:
            print('在%05d: 提前停止训练'% (self.stopped_epoch+1))
        

In [18]:
model = get_model()
_ = model.fit(x_train, y_train,
          batch_size=64,
          steps_per_epoch=5,
          epochs=30,
          verbose=0,
          callbacks=[LossAndErrorPrintingCallback(), EarlyStoppingAtMinLoss()])

For batch 0, loss is   23.36.
For batch 1, loss is 1043.09.
For batch 2, loss is   25.19.
For batch 3, loss is    8.33.
For batch 4, loss is    6.51.
The average loss for epoch 0 is  221.30 and mean absolute error is    8.41.
For batch 0, loss is   10.12.
For batch 1, loss is    6.01.
For batch 2, loss is    7.86.
For batch 3, loss is    6.52.
For batch 4, loss is    5.53.
The average loss for epoch 1 is    7.21 and mean absolute error is    2.26.
For batch 0, loss is    4.40.
For batch 1, loss is    4.59.
For batch 2, loss is    5.08.
For batch 3, loss is    4.74.
For batch 4, loss is    6.71.
The average loss for epoch 2 is    5.10 and mean absolute error is    1.89.
For batch 0, loss is    5.96.
For batch 1, loss is    4.13.
For batch 2, loss is    5.94.
For batch 3, loss is    7.71.
For batch 4, loss is   16.43.
The average loss for epoch 3 is    8.03 and mean absolute error is    2.30.
导入当前最佳模型
在00004: 提前停止训练


### 自定义学习率
在模型训练中通常要做的一件事是随着训练轮次改变学习率。Keras后端公开了可用于设置变量的get_value API。在此示例中，我们展示了如何使用自定义的回调来动态更改学习率。

注：这只是示例实现，请参见callbacks.LearningRateScheduler和keras.optimizers.schedules有关更一般的实现。

In [19]:
class LearningRateScheduler(tf.keras.callbacks.Callback):
    def __init__(self, schedule):
        super(LearningRateScheduler, self).__init__()
        self.schedule = schedule
        
    def on_epoch_begin(self, epoch, logs=None):
        if not hasattr(self.model.optimizer, 'lr'):
            raise ValueError('Optimizer没有lr参数。')
        # 获取当前lr
        lr = float(tf.keras.backend.get_value(self.model.optimizer.lr))
        # 调整lr
        scheduled_lr = self.schedule(epoch, lr)
        tf.keras.backend.set_value(self.model.optimizer.lr, scheduled_lr)
        print('Epoch %05d: 学习率为%6.4f.'%(epoch, scheduled_lr))

按轮次调整学习率

In [20]:
LR_SCHEDULE = [
    # (epoch to start, learning rate) tuples
    (3, 0.05), (6, 0.01), (9, 0.005), (12, 0.001)
]

def lr_schedule(epoch, lr):
    if epoch < LR_SCHEDULE[0][0] or epoch > LR_SCHEDULE[-1][0]:
        return lr
    for i in range(len(LR_SCHEDULE)):
        if epoch == LR_SCHEDULE[i][0]:
            return LR_SCHEDULE[i][1]
    return lr

model = get_model()
_ = model.fit(x_train, y_train,
          batch_size=64,
          steps_per_epoch=5,
          epochs=15,
          verbose=0,
          callbacks=[LossAndErrorPrintingCallback(), LearningRateScheduler(lr_schedule)])



Epoch 00000: 学习率为0.1000.
For batch 0, loss is   24.55.
For batch 1, loss is 1134.79.
For batch 2, loss is   26.95.
For batch 3, loss is    8.14.
For batch 4, loss is    6.28.
The average loss for epoch 0 is  240.14 and mean absolute error is    8.91.
Epoch 00001: 学习率为0.1000.
For batch 0, loss is    7.12.
For batch 1, loss is    7.65.
For batch 2, loss is    5.11.
For batch 3, loss is    5.36.
For batch 4, loss is    5.43.
The average loss for epoch 1 is    6.13 and mean absolute error is    2.06.
Epoch 00002: 学习率为0.1000.
For batch 0, loss is    5.18.
For batch 1, loss is    5.11.
For batch 2, loss is    5.08.
For batch 3, loss is    4.34.
For batch 4, loss is    4.21.
The average loss for epoch 2 is    4.78 and mean absolute error is    1.77.
Epoch 00003: 学习率为0.0500.
For batch 0, loss is    5.30.
For batch 1, loss is    3.95.
For batch 2, loss is    4.07.
For batch 3, loss is    3.63.
For batch 4, loss is    4.44.
The average loss for epoch 3 is    4.28 and mean absolute error is    1.